# Experiment 2: Conditional DCGAN with Experience Replay

In [ ]:
class ReplayBuffer:
    def __init__(self, max_length=1_000_000):
        self.buffer = deque(maxlen=max_length)
        self.memory = namedtuple('Experience', ['state', 'action', 'reward', 'next_state', 'done'])

    def __len__(self):
        return len(self.buffer)

    def append(self, experience):
        self.buffer.append(experience)

    def sample(self, batch_size):
        # sampled_exp contains a namedtuple
        sampled_exp = random.sample(self.buffer, batch_size)
        states, actions, rewards, next_states, dones = zip(*sampled_exp)

        return (
            np.squeeze(states),
            np.array(actions),
            np.array(rewards),
            np.squeeze(next_states),
            np.array(dones, dtype=np.bool),
        )